In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
class Layer:
    def __init__(self):
        self.input = None
        self.output = None
    
    def forward_propogation(self, input):
        raise NotImplementedError
    
    def backward_propogation(self, output_error, learning_rate):
        raise NotImplementedError

In [3]:
class FCLayer(Layer):
    def __init__(self, input_size, output_size):
        self.weights = np.random.rand(input_size, output_size) - 0.5
        self.bias = np.random.rand(1, output_size) - 0.5
        
    def forward_propogation(self, input_data):
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias
        return self.output
    
    def backward_propogation(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)
        
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error
        return input_error

In [4]:
class ActivationLayer(Layer):
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime
        
    def forward_propogation(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output
    
    def backward_propogation(self, output_error, learning_rate):
        return self.activation_prime(self.input) * output_error
    

In [5]:
# class ActivationFunctions:
    
#     @staticmethod
#     def relu(z):
#         a = np.maximum(0,z)
#         return a
    
#     @staticmethod
#     def sigmoid(z):
#         return 1/(1+np.exp(-z))
    
#     @staticmethod
#     def tanh(z):
#         return np.tanh(z) 
    
#     @staticmethod
#     def sigmoid_derivative(dA, z):
#         sig = sigmoid(z)
#         return dA * sig * (1 - sig)
    
#     @staticmethod
#     def relu_derivative(self,z):
#             return 0 if z < 0 else 1
    
#     @staticmethod
#     def tanh_derivative(z):
#        return 1-np.tanh(z)**2

In [6]:
def tanh(z):
    return np.tanh(z) 
    
def tanh_prime(z):
    return 1-np.tanh(z)**2

In [7]:
def mse(y_true, y_pred):
#
    return np.mean(np.power(y_true-y_pred, 2))
    #return np.sum(np.nan_to_num(-y*np.log(x)-(1-y)*np.log(1-x)))

def mse_prime(y_true, y_pred):
#def mse_prime(x, y):
    return 2*(y_pred-y_true)/y_true.size
    #return (x-y)

In [8]:
class NeuralNetwork:
    def __init__(self):
        self.layers = []
        self.loss = []
        self.loss_prime = None
    
    def add(self, layer):
        self.layers.append(layer)
    
    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime
    
    def predict(self, input_data):
        samples = len(input_data)
        result = []
        
        for i in range(samples):
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_propogation(output)
            result.append(output)
        
        return result
    
    def fit(self, x_train, y_train, epochs, learning_rate):
        samples = len(x_train)
        
        for i in range(epochs):
            err = 0
            for j in range(samples):
                output = x_train[j]
                for layer in self.layers:
                    output = layer.forward_propogation(output)
                
                err += self.loss(y_train[j], output)
                
                error = self.loss_prime(y_train[j], output)
                for layer in reversed(self.layers):
                    error = layer.backward_propogation(error, learning_rate)
                
            err /= samples
            print(f"Epoch: {i+1}/{epochs} Error: {err}")

In [9]:
x_train = np.array([[[0,0]], [[0,1]], [[1,0]], [[1,1]]])
y_train = np.array([[[0]], [[1]], [[1]], [[0]]])

In [10]:
net = NeuralNetwork()
net.add(FCLayer(2, 3))
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(3, 1))
net.add(ActivationLayer(tanh, tanh_prime))

In [11]:
net.use(mse, mse_prime)

In [12]:
net.fit(x_train, y_train, epochs=1000, learning_rate=0.1)

Epoch: 1/1000 Error: 0.4143458881842588
Epoch: 2/1000 Error: 0.3192084822563885
Epoch: 3/1000 Error: 0.3040783708792536
Epoch: 4/1000 Error: 0.2993216377119285
Epoch: 5/1000 Error: 0.29696781894349356
Epoch: 6/1000 Error: 0.2954223887197618
Epoch: 7/1000 Error: 0.294233013115354
Epoch: 8/1000 Error: 0.2932344800879229
Epoch: 9/1000 Error: 0.29235299103598955
Epoch: 10/1000 Error: 0.29154991645533745
Epoch: 11/1000 Error: 0.29080236212615695
Epoch: 12/1000 Error: 0.2900953309394019
Epoch: 13/1000 Error: 0.28941815327177245
Epoch: 14/1000 Error: 0.28876270826299966
Epoch: 15/1000 Error: 0.28812247470033225
Epoch: 16/1000 Error: 0.28749199434000655
Epoch: 17/1000 Error: 0.28686655136727746
Epoch: 18/1000 Error: 0.28624197037996957
Epoch: 19/1000 Error: 0.2856144823904702
Epoch: 20/1000 Error: 0.2849806318369791
Epoch: 21/1000 Error: 0.28433720968118725
Epoch: 22/1000 Error: 0.28368120405004044
Epoch: 23/1000 Error: 0.2830097633375968
Epoch: 24/1000 Error: 0.2823201686152893
Epoch: 25/1000

Epoch: 231/1000 Error: 0.004405009725275423
Epoch: 232/1000 Error: 0.004335899476862049
Epoch: 233/1000 Error: 0.004268704153624596
Epoch: 234/1000 Error: 0.004203349400249807
Epoch: 235/1000 Error: 0.0041397645248210735
Epoch: 236/1000 Error: 0.0040778822822822225
Epoch: 237/1000 Error: 0.004017638672708492
Epoch: 238/1000 Error: 0.003958972753243347
Epoch: 239/1000 Error: 0.0039018264626565474
Epoch: 240/1000 Error: 0.0038461444575674047
Epoch: 241/1000 Error: 0.003791873959457643
Epoch: 242/1000 Error: 0.003738964611670368
Epoch: 243/1000 Error: 0.0036873683456587316
Epoch: 244/1000 Error: 0.003637039255807648
Epoch: 245/1000 Error: 0.0035879334822067122
Epoch: 246/1000 Error: 0.003540009100803036
Epoch: 247/1000 Error: 0.003493226020407802
Epoch: 248/1000 Error: 0.0034475458860721707
Epoch: 249/1000 Error: 0.0034029319883863655
Epoch: 250/1000 Error: 0.003359349178290208
Epoch: 251/1000 Error: 0.0033167637870155503
Epoch: 252/1000 Error: 0.003275143550809675
Epoch: 253/1000 Error: 

Epoch: 422/1000 Error: 0.0009581033730481338
Epoch: 423/1000 Error: 0.0009538161599249469
Epoch: 424/1000 Error: 0.0009495645008645243
Epoch: 425/1000 Error: 0.0009453479689423166
Epoch: 426/1000 Error: 0.0009411661439175398
Epoch: 427/1000 Error: 0.0009370186121045191
Epoch: 428/1000 Error: 0.0009329049662469599
Epoch: 429/1000 Error: 0.0009288248053950729
Epoch: 430/1000 Error: 0.0009247777347854871
Epoch: 431/1000 Error: 0.0009207633657238578
Epoch: 432/1000 Error: 0.0009167813154701292
Epoch: 433/1000 Error: 0.0009128312071263626
Epoch: 434/1000 Error: 0.0009089126695270568
Epoch: 435/1000 Error: 0.0009050253371319001
Epoch: 436/1000 Error: 0.0009011688499209491
Epoch: 437/1000 Error: 0.0008973428532920601
Epoch: 438/1000 Error: 0.000893546997960607
Epoch: 439/1000 Error: 0.0008897809398614006
Epoch: 440/1000 Error: 0.0008860443400527419
Epoch: 441/1000 Error: 0.0008823368646225783
Epoch: 442/1000 Error: 0.0008786581845966612
Epoch: 443/1000 Error: 0.0008750079758487537
Epoch: 444/

Epoch: 684/1000 Error: 0.00042583591405133257
Epoch: 685/1000 Error: 0.00042489439464867186
Epoch: 686/1000 Error: 0.0004239567866407423
Epoch: 687/1000 Error: 0.00042302306631471903
Epoch: 688/1000 Error: 0.00042209321014631106
Epoch: 689/1000 Error: 0.00042116719479787603
Epoch: 690/1000 Error: 0.00042024499711661417
Epoch: 691/1000 Error: 0.0004193265941327513
Epoch: 692/1000 Error: 0.0004184119630577458
Epoch: 693/1000 Error: 0.00041750108128253623
Epoch: 694/1000 Error: 0.000416593926375774
Epoch: 695/1000 Error: 0.0004156904760821219
Epoch: 696/1000 Error: 0.00041479070832052447
Epoch: 697/1000 Error: 0.00041389460118254563
Epoch: 698/1000 Error: 0.000413002132930677
Epoch: 699/1000 Error: 0.0004121132819967068
Epoch: 700/1000 Error: 0.00041122802698008193
Epoch: 701/1000 Error: 0.00041034634664630527
Epoch: 702/1000 Error: 0.0004094682199253399
Epoch: 703/1000 Error: 0.00040859362591003177
Epoch: 704/1000 Error: 0.00040772254385455553
Epoch: 705/1000 Error: 0.0004068549531728806

Epoch: 908/1000 Error: 0.0002822240412627625
Epoch: 909/1000 Error: 0.0002817905839942415
Epoch: 910/1000 Error: 0.0002813583844346441
Epoch: 911/1000 Error: 0.00028092743724294876
Epoch: 912/1000 Error: 0.0002804977371079373
Epoch: 913/1000 Error: 0.00028006927874798864
Epoch: 914/1000 Error: 0.00027964205691087475
Epoch: 915/1000 Error: 0.00027921606637355844
Epoch: 916/1000 Error: 0.00027879130194199427
Epoch: 917/1000 Error: 0.00027836775845092523
Epoch: 918/1000 Error: 0.00027794543076368875
Epoch: 919/1000 Error: 0.00027752431377202265
Epoch: 920/1000 Error: 0.00027710440239586426
Epoch: 921/1000 Error: 0.00027668569158316366
Epoch: 922/1000 Error: 0.0002762681763096889
Epoch: 923/1000 Error: 0.0002758518515788417
Epoch: 924/1000 Error: 0.00027543671242145754
Epoch: 925/1000 Error: 0.0002750227538956405
Epoch: 926/1000 Error: 0.00027460997108655154
Epoch: 927/1000 Error: 0.0002741983591062474
Epoch: 928/1000 Error: 0.00027378791309348656
Epoch: 929/1000 Error: 0.00027337862821355

In [13]:
print(net.predict(x_train))

[array([[0.00083719]]), array([[0.97865942]]), array([[0.97699821]]), array([[-0.00138222]])]


In [14]:
# load the dataset
data = pd.read_csv("data_banknote_authentication.txt", header=None)
# split into input and output columns
X, Y = data.values[:, :-1], data.values[:, -1] #separate data into input and output features
# ensure all data are floating point values

X = X.astype('float32')

# encode strings to integer
Y = LabelEncoder().fit_transform(Y)
# split into train and test datasets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33) #split data into train and test sets in 80-20 ratio
# determine the number of input features
n_features = X.shape[1]

In [15]:
X = []
for i in range(X_train.shape[0]):
    inner_list = []
    outer_list = []
    inner_list.append(X_train[i])
    X.append(inner_list)

In [16]:
XT = np.asarray(X)

In [17]:
YT = Y_train.reshape(Y_train.size, 1,1)

In [18]:
nn = NeuralNetwork()
nn.add(FCLayer(4, 3))
nn.add(ActivationLayer(tanh, tanh_prime))
nn.add(FCLayer(3, 1))
nn.add(ActivationLayer(tanh, tanh_prime))

In [19]:
nn.use(mse, mse_prime)

In [20]:
nn.fit(XT, YT, epochs=1000, learning_rate=0.1)

Epoch: 1/1000 Error: 0.04198270902042572
Epoch: 2/1000 Error: 0.03265929455191658
Epoch: 3/1000 Error: 0.029146932736910342
Epoch: 4/1000 Error: 0.007735306401653901
Epoch: 5/1000 Error: 0.0013610242563605168
Epoch: 6/1000 Error: 0.0007922450239114931
Epoch: 7/1000 Error: 0.0005893134480921344
Epoch: 8/1000 Error: 0.00048096341778467984
Epoch: 9/1000 Error: 0.0004128222985760378
Epoch: 10/1000 Error: 0.0003656005802565008
Epoch: 11/1000 Error: 0.00033014832800056085
Epoch: 12/1000 Error: 0.0003020700737361505
Epoch: 13/1000 Error: 0.000279093229381459
Epoch: 14/1000 Error: 0.0002598584735386321
Epoch: 15/1000 Error: 0.000243469375730475
Epoch: 16/1000 Error: 0.0002293029579996963
Epoch: 17/1000 Error: 0.00021691070074064583
Epoch: 18/1000 Error: 0.00020596051163641657
Epoch: 19/1000 Error: 0.00019620065546568037
Epoch: 20/1000 Error: 0.00018743643431573705
Epoch: 21/1000 Error: 0.00017951462712615147
Epoch: 22/1000 Error: 0.00017231281801524561
Epoch: 23/1000 Error: 0.00016573189708061

Epoch: 185/1000 Error: 2.7368768269194234e-05
Epoch: 186/1000 Error: 2.7238718549445407e-05
Epoch: 187/1000 Error: 2.7109945536711693e-05
Epoch: 188/1000 Error: 2.698242949517385e-05
Epoch: 189/1000 Error: 2.6856151089298572e-05
Epoch: 190/1000 Error: 2.6731091373708254e-05
Epoch: 191/1000 Error: 2.660723178335405e-05
Epoch: 192/1000 Error: 2.6484554123985516e-05
Epoch: 193/1000 Error: 2.6363040562904515e-05
Epoch: 194/1000 Error: 2.624267361999023e-05
Epoch: 195/1000 Error: 2.6123436158991387e-05
Epoch: 196/1000 Error: 2.6005311379073946e-05
Epoch: 197/1000 Error: 2.588828280661542e-05
Epoch: 198/1000 Error: 2.5772334287238495e-05
Epoch: 199/1000 Error: 2.5657449978076357e-05
Epoch: 200/1000 Error: 2.554361434025876e-05
Epoch: 201/1000 Error: 2.5430812131616503e-05
Epoch: 202/1000 Error: 2.531902839959448e-05
Epoch: 203/1000 Error: 2.5208248474364253e-05
Epoch: 204/1000 Error: 2.5098457962133797e-05
Epoch: 205/1000 Error: 2.4989642738645592e-05
Epoch: 206/1000 Error: 2.488178894285835

Epoch: 365/1000 Error: 1.4599085805976914e-05
Epoch: 366/1000 Error: 1.4558609930146439e-05
Epoch: 367/1000 Error: 1.4518311005935346e-05
Epoch: 368/1000 Error: 1.4478188166750992e-05
Epoch: 369/1000 Error: 1.4438240575829369e-05
Epoch: 370/1000 Error: 1.439846742584793e-05
Epoch: 371/1000 Error: 1.4358867938503302e-05
Epoch: 372/1000 Error: 1.431944136405591e-05
Epoch: 373/1000 Error: 1.4280186980833998e-05
Epoch: 374/1000 Error: 1.4241104094704019e-05
Epoch: 375/1000 Error: 1.4202192038501309e-05
Epoch: 376/1000 Error: 1.4163450171421228e-05
Epoch: 377/1000 Error: 1.4124877878373859e-05
Epoch: 378/1000 Error: 1.4086474569296197e-05
Epoch: 379/1000 Error: 1.4048239678429152e-05
Epoch: 380/1000 Error: 1.4010172663552902e-05
Epoch: 381/1000 Error: 1.3972273005185383e-05
Epoch: 382/1000 Error: 1.3934540205741865e-05
Epoch: 383/1000 Error: 1.38969737886592e-05
Epoch: 384/1000 Error: 1.3859573297484612e-05
Epoch: 385/1000 Error: 1.3822338294928606e-05
Epoch: 386/1000 Error: 1.3785268361886

Epoch: 545/1000 Error: 9.631826635809628e-06
Epoch: 546/1000 Error: 9.613430959737426e-06
Epoch: 547/1000 Error: 9.595087879508734e-06
Epoch: 548/1000 Error: 9.576796564176307e-06
Epoch: 549/1000 Error: 9.558556179102848e-06
Epoch: 550/1000 Error: 9.540365885724865e-06
Epoch: 551/1000 Error: 9.522224841310814e-06
Epoch: 552/1000 Error: 9.504132198715388e-06
Epoch: 553/1000 Error: 9.486087106130648e-06
Epoch: 554/1000 Error: 9.46808870683099e-06
Epoch: 555/1000 Error: 9.45013613891768e-06
Epoch: 556/1000 Error: 9.432228535055082e-06
Epoch: 557/1000 Error: 9.414365022207552e-06
Epoch: 558/1000 Error: 9.39654472136784e-06
Epoch: 559/1000 Error: 9.378766747285368e-06
Epoch: 560/1000 Error: 9.361030208189195e-06
Epoch: 561/1000 Error: 9.343334205507808e-06
Epoch: 562/1000 Error: 9.325677833584637e-06
Epoch: 563/1000 Error: 9.30806017939101e-06
Epoch: 564/1000 Error: 9.290480322234819e-06
Epoch: 565/1000 Error: 9.272937333467217e-06
Epoch: 566/1000 Error: 9.255430276184333e-06
Epoch: 567/100

Epoch: 728/1000 Error: 5.8550609395052026e-06
Epoch: 729/1000 Error: 5.8260546271126036e-06
Epoch: 730/1000 Error: 5.797087526825397e-06
Epoch: 731/1000 Error: 5.768169396605714e-06
Epoch: 732/1000 Error: 5.7393100048094625e-06
Epoch: 733/1000 Error: 5.7105191044211275e-06
Epoch: 734/1000 Error: 5.6818064071043685e-06
Epoch: 735/1000 Error: 5.653181557226174e-06
Epoch: 736/1000 Error: 5.624654106015712e-06
Epoch: 737/1000 Error: 5.596233486012855e-06
Epoch: 738/1000 Error: 5.567928985971943e-06
Epoch: 739/1000 Error: 5.5397497263724165e-06
Epoch: 740/1000 Error: 5.511704635689498e-06
Epoch: 741/1000 Error: 5.483802427573906e-06
Epoch: 742/1000 Error: 5.4560515790787825e-06
Epoch: 743/1000 Error: 5.428460310063219e-06
Epoch: 744/1000 Error: 5.401036563895991e-06
Epoch: 745/1000 Error: 5.373787989566859e-06
Epoch: 746/1000 Error: 5.346721925303165e-06
Epoch: 747/1000 Error: 5.319845383775767e-06
Epoch: 748/1000 Error: 5.293165038963283e-06
Epoch: 749/1000 Error: 5.266687214729228e-06
Epo

Epoch: 909/1000 Error: 3.2463586944967046e-06
Epoch: 910/1000 Error: 3.2408602327909915e-06
Epoch: 911/1000 Error: 3.2353995287528217e-06
Epoch: 912/1000 Error: 3.2299761548942725e-06
Epoch: 913/1000 Error: 3.2245896898654485e-06
Epoch: 914/1000 Error: 3.2192397183498258e-06
Epoch: 915/1000 Error: 3.2139258309607405e-06
Epoch: 916/1000 Error: 3.208647624141377e-06
Epoch: 917/1000 Error: 3.203404700065301e-06
Epoch: 918/1000 Error: 3.19819666654002e-06
Epoch: 919/1000 Error: 3.1930231369118003e-06
Epoch: 920/1000 Error: 3.1878837299729526e-06
Epoch: 921/1000 Error: 3.182778069870446e-06
Epoch: 922/1000 Error: 3.177705786016561e-06
Epoch: 923/1000 Error: 3.172666513001086e-06
Epoch: 924/1000 Error: 3.167659890505753e-06
Epoch: 925/1000 Error: 3.162685563219764e-06
Epoch: 926/1000 Error: 3.1577431807565895e-06
Epoch: 927/1000 Error: 3.1528323975743813e-06
Epoch: 928/1000 Error: 3.147952872894812e-06
Epoch: 929/1000 Error: 3.143104270626107e-06
Epoch: 930/1000 Error: 3.1382862592863394e-06

In [21]:
nn.predict(XT)

[array([[0.99999432]]),
 array([[0.99999787]]),
 array([[0.99986549]]),
 array([[0.00048451]]),
 array([[0.00060108]]),
 array([[0.99999793]]),
 array([[0.99999809]]),
 array([[0.99888023]]),
 array([[0.00014952]]),
 array([[-0.00071847]]),
 array([[0.99739084]]),
 array([[0.99999806]]),
 array([[0.00049431]]),
 array([[-0.00088528]]),
 array([[-0.00070983]]),
 array([[0.99737103]]),
 array([[0.00031364]]),
 array([[0.00059676]]),
 array([[0.99999767]]),
 array([[0.99779144]]),
 array([[0.99911323]]),
 array([[0.99855518]]),
 array([[0.00056795]]),
 array([[0.99743495]]),
 array([[0.99999807]]),
 array([[0.0006015]]),
 array([[0.9985707]]),
 array([[0.00022127]]),
 array([[0.00054527]]),
 array([[0.99998647]]),
 array([[0.99849554]]),
 array([[0.00057586]]),
 array([[0.99736407]]),
 array([[0.00058747]]),
 array([[0.99861738]]),
 array([[0.99999819]]),
 array([[0.99729469]]),
 array([[0.0003598]]),
 array([[0.00058294]]),
 array([[0.00056751]]),
 array([[0.00051022]]),
 array([[0.00060